# Logit Response Dynamics

**Yuya Furusawa**

*Graduate School of Economics, University of Tokyo*

This notebook demonstrates how to study logit best response dynamics model using the logitdyn Julia library.

In [1]:
using GameTheory
using Plots
plotly()

Plots.PlotlyBackend()

## Logit Response Dynamics

Logit response dynamics model is one of the learning models.
There is much literature that studies the learning in games, for example, fictitious play, (myopic) best response dynamics, Kandori-Mailath-Rob, sampling best response models. Please refer these notes:

- fictitious play
- best response dynamics

The learning model with random mistakes, including KMR model, is criticised by the absense of the foundation of mistakes.

In contrast, **Logit response dynamics**, which also incorporates the random mistakes, is based on a theory about the origin of mistakes.

### Model

Let $\Gamma = (I, {(S_i, u_i)}_{i \in I})$ be a finite normal form game.
- $I=\{1,\ldots,N\}$ : player set
- $S_i$ : strategies set ($S$ : pure strategy profile)
- $u_i$ : payoff function

Denote $s=(s_i, s_{-i}) \in S$ and $u_i(s_i, s_{-i})=u_i(s)$.

Players behave as myopic best replier, but have mistakes in thier decisions.
A player is chosen to update her action given other players' actions every period (**asynchronous learning**).

When player $i$ is chosen, she will update her action to $s_i$ given actions $s_{-i}$.
The probability of choosing action $s_i$ is given by **logit choice function** :
$$
    p_i(s_i, s_{-i}) = \frac{\exp(\beta u_i(s_i, s_{-i}))}{\sum_{s_i' \in S_i} \exp(\beta u_i(s_i', s_{-i}))}
$$
where $0 < \beta < \infty$.

$\beta$ can be interpreted as an inverse measure of the level of noise in players' decisions.
As $\beta$ becomes larger, the size of noise becomes smaller.
From the microeconomic point of view, this modeling is based on a random-utility model, decision thoretic foundation.

Logit response dynamics can converge to the set of Nash equilibria in the potential game.
First, we introduce the definition of potential game.

**Definition(potential game)**

The $\Gamma$ is a *potential game* if there exists a function $\rho:S \rightarrow \mathbb{R}$ such that for all $i \in I$, $s_i, s_i' \in S_i$, $s_{-i} \in S_{-i}$,
$$
    u_i(s_i, s_{-i}) - u_i(s_i', s_{-i}) = \rho(s_i, s_{-i}) - \rho(s_i', s_{-i})
$$

$\rho$ is called the potential.

The following game is the potential game:

1$\backslash$2 | $A$           | $B$ 
 ------------- |---------------| ---------
 $A$           | 4, 4          |   0, 3   
 $B$           | 3, 0          |   2, 2     
 
To see this, the potential $\rho$ can be defined

1$\backslash$2 | $A$        | $B$ 
 ------------- |------------| ---------
 $A$           | 1          |   0  
 $B$           | 0          |   2     

(Note every $2 \times 2$ symmetric game is the potential game)

**Theorem**

In the potential game, the set of stable states of logit response dynamics is equal to the set of maximizers of  𝜌 .

We will check this later.

## `logitdyn.jl` usage

To create LogitDynamics class instance, you need the played game and the parameter $\beta$.
The game has to be NormalFormGame instance.
As argued before, $\beta$ is the level of noise in the players' decisions.

In [2]:
payoff = [4 0;
          3 2]
game = NormalFormGame(payoff)
beta = 2.0
ldy = LogitDynamics(game, beta)

LogitDynamics{2, Int64, Float64}((2×2 Player{2, Int64}:
 4  0
 3  2, 2×2 Player{2, Int64}:
 4  0
 3  2), (2, 2), 2.0, Array[[1.0 1.1353352832366128; 0.01831563888873418 1.0183156388887342], [1.0 1.1353352832366128; 0.01831563888873418 1.0183156388887342]])

In [3]:
ldy.choice_probs

2-element Vector{Array}:
 [1.0 1.1353352832366128; 0.01831563888873418 1.0183156388887342]
 [1.0 1.1353352832366128; 0.01831563888873418 1.0183156388887342]

### `play` method

`play` method returns a new action profile in the model.
You can specify the initial action profile.
In addition, you can proceed the dynamics multiple times by specifying num_reps, which is set to 1 by default.

In [4]:
init_actions = (1, 1)
play(ldy, init_actions)

2-element Vector{Int64}:
 1
 1

### `time_series` method

`time_series` method returns a time series of action distributions.
You have to specify the length of time series.
In addition, you can choose initial action profile from which the dynamics proceeds.

In [5]:
ts_length = 5
init_actions = (1, 1)
time_series(ldy, ts_length, init_actions)

2×5 Matrix{Int64}:
 1  1  1  1  1
 1  1  1  2  1

## Example

We see the convergence of the logit response dynamics in the potential game.

In [6]:
payoff = [4 0; 3 2]
game = NormalFormGame(payoff)
beta = 4.0
ldy = LogitDynamics(game, beta)

LogitDynamics{2, Int64, Float64}((2×2 Player{2, Int64}:
 4  0
 3  2, 2×2 Player{2, Int64}:
 4  0
 3  2), (2, 2), 4.0, Array[[1.0 1.0183156388887342; 0.00033546262790251185 1.0003354626279024], [1.0 1.0183156388887342; 0.00033546262790251185 1.0003354626279024]])

In [7]:
ts_length = 1000
init_actions = (1, 1)
series = time_series(ldy, ts_length, init_actions)

2×1000 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2
 1  1  1  1  1  1  1  1  1  1  1  1  1     2  2  2  2  2  2  2  2  2  2  2  2

In [8]:
seriesB_1 = [i==2 ? 1 : 0 for i in series[1,:]]
seriesB_2 = [i==2 ? 1 : 0 for i in series[2,:]]
freq1 = accumulate(+, seriesB_1) ./ collect(1:ts_length) 
freq2 = accumulate(+, seriesB_2) ./ collect(1:ts_length)
labels=["Player 1" "Player 2"]
plot([freq1, freq2], label=labels)

As you can see, (B,B) is a stable state of this dynamics, and this state maximizes the potential $\rho$.

Next, we simulate the logit reponse dynamics in the Young's game.
Young's game is defined as follows:

1$\backslash$2 | $A$           | $B$      | $C$
 ------------- |---------------| ---------| --------
 $A$           | 6, 6          |   0, 0   |   0, 0
 $B$           | 5, 5          |   7, 7   |   5, 5
 $C$           | 0, 0          |   5, 5   |   8, 8

In [9]:
young_game = [6 0 0;
              5 7 5;
              0 5 8]
game = NormalFormGame(young_game)
beta = 4.0
ldy = LogitDynamics(game, beta)

LogitDynamics{2, Int64, Float64}((3×3 Player{2, Int64}:
 6  0  0
 5  7  5
 0  5  8, 3×3 Player{2, Int64}:
 6  0  0
 5  7  5
 0  5  8), (3, 3), 4.0, Array[[1.0 1.0183156388887342 1.0183156389264856; 6.914400106940203e-13 1.0000000000006914 1.0003354626285939; 1.2664165549094176e-14 6.144212365992376e-6 1.000006144212366], [1.0 1.0183156388887342 1.0183156389264856; 6.914400106940203e-13 1.0000000000006914 1.0003354626285939; 1.2664165549094176e-14 6.144212365992376e-6 1.000006144212366]])

In [10]:
ts_length = 1000
init_actions = (1, 1)
series = time_series(ldy, ts_length, init_actions)
seriesB_1 = [i==2 ? 1 : 0 for i in series[1,:]]
seriesB_2 = [i==2 ? 1 : 0 for i in series[2,:]]
freq1 = accumulate(+, seriesB_1) ./ collect(1:ts_length) 
freq2 = accumulate(+, seriesB_2) ./ collect(1:ts_length)
labels=["Player 1" "Player 2"]
plot([freq1, freq2], label=labels)

---

## References

- Alós-Ferrer, Carlos, and Nick Netzer. "The logit-response dynamics." Games and Economic Behavior 68.2 (2010): 413-427.